<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [26]:
#Start Here
import numpy as np
import pandas as pd
import gensim
import os, sys, re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim

import spacy
nlp = spacy.load('en_core_web_lg')

In [2]:
df = pd.read_csv("./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

In [3]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [8]:
tokenized_reviews = [[token for token in simple_preprocess(review) if token not in STOPWORDS] \
                      for review in df['reviews.text']]
df['reviews.tokenized'] = pd.Series(tokenized_reviews)

In [9]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,reviews.tokenized
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, quality, missing, backup, s..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","[bulk, expensive, way, products, like]"
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...","[duracell, price, happy]"
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...","[work, brand, batteries, better, price]"
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...","[batteries, long, lasting, price, great]"


In [24]:
def get_reviews(df, token_col):
    """
    Generator function that takes a dataframe with a tokenized column and yields a tokenized 
    row for processing
    
    Parameters:
    -----------
    df: dataframe
    taken_col: name of the column of tokenized text
    
    Yields:
    -------
    review_token: one record of tokenized review
    """
    assert token_col in df.columns, f"{token_col} does not exist!"
    
    for index, row in df.iterrows():
        review_token = row[token_col]
        yield review_token

In [16]:
# construct a dictionary of tokenized words with their respective ids
id2word = corpora.Dictionary(get_reviews(df, 'reviews.tokenized'))

In [34]:
# check the id of word 'brand'
print(f'The id for word "brand" is {id2word.token2id["brand"]}.')
print(f'The size of the id2word dictionary is {sys.getsizeof(id2word)}.')
print(f'There are {len(id2word.keys())} unique words in the dictionary.')

The id for word "brand" is 21.
The size of the id2word dictionary is 56.
There are 9621 unique words in the dictionary.


In [36]:
# get LDA representation of the words
corpus = [id2word.doc2bow(review) for review in get_reviews(df, 'reviews.tokenized')]

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=45, # random state for multicore doesnt work in gensim
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [38]:
lda.print_topics()

[(0,
  '0.091*"old" + 0.073*"year" + 0.058*"loves" + 0.042*"bought" + 0.022*"use" + 0.022*"tablet" + 0.018*"games" + 0.015*"granddaughter" + 0.015*"grandson" + 0.014*"yr"'),
 (1,
  '0.100*"great" + 0.089*"good" + 0.084*"price" + 0.076*"batteries" + 0.035*"work" + 0.026*"long" + 0.025*"value" + 0.023*"buy" + 0.020*"quality" + 0.018*"brand"'),
 (2,
  '0.081*"battery" + 0.057*"life" + 0.052*"long" + 0.036*"time" + 0.018*"bought" + 0.015*"money" + 0.014*"longer" + 0.013*"lasts" + 0.011*"lasted" + 0.010*"hours"'),
 (3,
  '0.039*"tablet" + 0.035*"books" + 0.029*"great" + 0.022*"amazon" + 0.020*"reading" + 0.019*"read" + 0.019*"use" + 0.018*"easy" + 0.018*"games" + 0.016*"movies"'),
 (4,
  '0.052*"tablet" + 0.031*"good" + 0.023*"great" + 0.022*"amazon" + 0.019*"apps" + 0.016*"kids" + 0.016*"price" + 0.015*"ipad" + 0.011*"google" + 0.011*"time"'),
 (5,
  '0.088*"batteries" + 0.021*"amazon" + 0.013*"work" + 0.013*"duracell" + 0.012*"use" + 0.012*"battery" + 0.010*"long" + 0.010*"ve" + 0.009*"aa

In [43]:
# print the topics in a more readable manner
topics = [' '.join(re.findall(r'"([^"]*)"',t[1])) for t in lda.print_topics()]
print(*topics, sep = "\n\n") 

old year loves bought use tablet games granddaughter grandson yr

great good price batteries work long value buy quality brand

battery life long time bought money longer lasts lasted hours

tablet books great amazon reading read use easy games movies

tablet good great amazon apps kids price ipad google time

batteries amazon work duracell use battery long ve aa brand

great tablet gift bought loves christmas price product got works

kindle screen new like light love read reading reader better

needs laptop charging keyboard charger screen entertained great kids parents

love kids tablet bought games case play great kindle parental

works excellent buy charge better batteries fine time worked fast

easy use tablet great recommend love perfect size kids product

apps kid friendly loves lots kids tablet ok user games

like batteries amazon need use box ve come came store

device tablet nephew amazon good account advertised ads entertainment little


In [44]:
#Visualize LDA
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

c:\users\han-chung lee\.virtualenvs\ds-unit-4-sprint-1-nlp-o1ln3r8m\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.239226  0.005930       1        1  12.045135
1     -0.211420  0.129283       2        1  11.695461
4      0.061994 -0.056899       3        1  10.805236
3      0.129503 -0.049768       4        1   8.833228
7      0.014601 -0.127706       5        1   7.574609
11     0.181128  0.014583       6        1   6.817101
9      0.172888 -0.009144       7        1   6.355785
0      0.186235  0.138988       8        1   6.254967
6      0.126546  0.193048       9        1   6.125741
13    -0.149996 -0.041587      10        1   5.073088
12     0.106579  0.002606      11        1   4.012749
10    -0.200188  0.026214      12        1   3.992630
2     -0.153877  0.104940      13        1   3.923933
8     -0.009421 -0.164984      14        1   3.647171
14    -0.015346 -0.165504      15        1   2.843156, topic_info=     Category         Freq       Term        Total  loglift  logprob
19    Default  6845.000000  batteries  6845.000000  30.0000  30.0000
22    Default  8811.000000      great  8811.000000  29.0000  29.0000
3156  Default  3097.000000     kindle  3097.000000  28.0000  28.0000
605   Default  2323.000000        old  2323.000000  27.0000  27.0000
4196  Default  6780.000000     tablet  6780.000000  26.0000  26.0000
...       ...          ...        ...          ...      ...      ...
99    Topic15    56.953747      works  1416.797241   0.3463  -5.0710
1075  Topic15    49.630043       free   619.604858   1.0358  -5.2086
1844  Topic15    47.867214      music   439.407806   1.3433  -5.2448
24    Topic15    53.892326       long  2243.920166  -0.1687  -5.1262
2245  Topic15    49.463470     screen  1657.231079   0.0486  -5.2120

[1161 rows x 6 columns], token_table=      Topic      Freq Term
term                      
58        1  0.778790   aa
58        2  0.164411   aa
58       10  0.038939   aa
58       12  0.015143   aa
58       13  0.002163   aa
...     ...       ...  ...
1566      9  0.098824  yrs
1566     11  0.815300  yrs
6431      9  0.096562  äôs
6431     15  0.893198  äôs
8502     12  0.867619   äù

[3799 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 5, 4, 8, 12, 10, 1, 7, 14, 13, 11, 3, 9, 15])

In [45]:
# Overall Model / Documents
print(lda[corpus[0]])
distro = [lda[d] for d in corpus]
print(distro[0])
def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

[(1, 0.2980535), (8, 0.62964815)]
[(1, 0.29807696), (8, 0.6296247)]


In [46]:
new_distro[:5]

[{0: 0,
  1: 0.29807696,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0.6296247,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0},
 {0: 0.011111673,
  1: 0.011111703,
  2: 0.011111673,
  3: 0.011111684,
  4: 0.011111683,
  5: 0.011111704,
  6: 0.011111672,
  7: 0.011111684,
  8: 0.011111676,
  9: 0.011111668,
  10: 0.011111691,
  11: 0.011111678,
  12: 0.011111676,
  13: 0.84443647,
  14: 0.011111685},
 {0: 0.016667224,
  1: 0.7666584,
  2: 0.016667275,
  3: 0.01666723,
  4: 0.016667236,
  5: 0.016667444,
  6: 0.016667312,
  7: 0.01666726,
  8: 0.016667223,
  9: 0.016667234,
  10: 0.016667232,
  11: 0.01666725,
  12: 0.016667224,
  13: 0.016667224,
  14: 0.016667224},
 {0: 0.011111191,
  1: 0.84444326,
  2: 0.011111194,
  3: 0.011111193,
  4: 0.011111197,
  5: 0.01111121,
  6: 0.011111197,
  7: 0.011111205,
  8: 0.011111191,
  9: 0.0111111915,
  10: 0.011111216,
  11: 0.011111193,
  12: 0.011111191,
  13: 0.011111194,
  14: 0.011111191},
 {0: 0.011111174,
  1: 0.8444435,
  

In [ ]:
df2 = pd.DataFrame.from_records(new_distro, index=titles)
df2.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [47]:
def compute_coherence_values(dictionary, corpus, df, token_col, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    tokens = list(get_reviews(df, token_col))
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            stream = get_reviews(df, token_col)
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
coherence_values = compute_coherence_values(dictionary=id2word, 
                                            corpus=corpus, 
                                            df=df, 
                                            token_col='reviews.tokenized', 
                                            start=2, 
                                            limit=40, 
                                            step=6,
                                            passes=20)

# Findings

There seem to be a lot of reviews that's centered around battery/batteries and kindle tablets. Also, some keywords are kindle, tablet, love, and price. Combined with the 15 topics that LDA chose, it seems Amazon's tablet is a hit from the first impression. And the other common product that people purchase are batteries. And as with normal consumer behavior, people care about price and value and tend to buy things as gift, especially during Christmas.

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling